# "This is my attempt to move my ET processing code to python so I can use the SSEBOP code" -Rebecca or Paul
### to install openet.ssebop use "pip install openet-ssebop" in cmd window

# Import Libraries

In [13]:
import datetime
import pprint

import ee
import pandas as pd

from IPython.display import Image
import openet.ssebop as model

#import geopandas as gpd #optional
#import folium #optional
ee.Authenticate()  #Kathryn had to add this command
ee.Initialize()   #use_cloud_api=True)

Enter verification code: 4/1AX4XfWjFcVWPlgEG4NQxIBtjdqejYPZ1pemf3udDQyiPgCSfeSr-V8B-k9A

Successfully saved authorization token.


# Functions

In [14]:
#function to make code output into a table
def makeTable (info):
    #return pd.DataFrame.from_dict(info.items())
    #return pd.DataFrame(info.items(),columns=['ET','Value'])
    return pd.DataFrame.from_dict(info,orient='index',columns=["et"])

In [15]:
#Old functions from running the code a different way
def reduceFunc(image):
    median = image.reduceRegion(**{
    'reducer': ee.Reducer.median(),
    'scale':30,
    'geometry': study_area,
    'maxPixels': 1e11
    }).get('et')
    
    newFeature = ee.Feature(None,{'et':median}).copyProperties(image,['system:time_start'])
    
    return newFeature


def clipImageCol(image):
    image1 = image.clip(study_area)
    return image1;
  

In [16]:
monthly_df = {}

# Is ee working?
### see result

In [17]:
#inserted to get ee to read

try:
    ee.Initialize()
    print('worked')
except ee.EEException:
    print('trying a second time')
    ee.Authenticate()
    ee.Initialize()

worked


# This code chunk runs the monthly SSEBOP model for the specified years and feature. The output is a csv file written to Drive with actual ET values.

## Need to change inputs in the first section of code and the file path in last line of this cell
## Read in polygon uses shapefile(s) in your GEE assets


In [23]:
###########################
#Inputs to change

study_crs = 'EPSG:32612'#'EPSG:6350' #'EPSG:5070'

#read in polygon
#feat1 = ee.FeatureCollection('users/kladig')
#featPoly = feat1.filter(ee.Filter.eq("Test5", 0))
featPoly = ee.FeatureCollection('users/kladig/Bryce/BryceStudyAreaRough')

#choose years for analysis
my_list = list(range(2000,2020))

#################
#other inputs that you don't need to change
study_area = featPoly
study_region = study_area


et_reference_source = 'IDAHO_EPSCOR/GRIDMET'
et_reference_band = 'etr'
et_reference_factor = 0.85
et_reference_resample = 'nearest'

cloud_cover = 70

interp_days = 32
# Interpolation method - currently only LINEAR is supported
interp_method = 'LINEAR'


###########################

#This is where the loop starts based on inputs. It takes 2+ hours to run for 2000-2020 for a given polygon. 


for item in my_list:
    yr = item 

    #if yr > 2013: #I took out L8 because I'm not sure that the model calibrates L7 and L8
    #2013-04-11T00:00:00 - 2020-10-02T00:00:00
    #    collections = ['LANDSAT/LC08/C01/T1_TOA'] 
    #elif yr <= 2013 and yr > 1999:
        #1999-01-01T00:00:00 - 2020-09-10T00:00:00
    #    collections = ['LANDSAT/LE07/C01/T1_TOA']#https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_TOA
    if yr > 1999:
        #1999-01-01T00:00:00 - 2020-09-10T00:00:00
        collections = ['LANDSAT/LE07/C01/T1_TOA']#https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_TOA
    else:
        #1984-01-01T00:00:00 - 2012-05-05T00:00:00
        collections = ['LANDSAT/LT05/C01/T1_TOA']

    for mo in range(1,13):
        start_date = f"{yr}-{mo:02}-01"
        if mo == 12:
            end_date = f"{yr+1}-01-01"
        else:
            end_date = f"{yr}-{mo+1:02}-01"
        
        model_obj = model.Collection(collections=collections, 
                                     et_reference_source=et_reference_source,
                                     et_reference_band=et_reference_band, 
                                     et_reference_factor=et_reference_factor,
                                     start_date=start_date,
                                     end_date=end_date, 
                                     geometry=study_area, #changed from test_point
                                     cloud_cover_max=70)

        monthly_coll = model_obj.interpolate(t_interval='monthly', variables=['et', 
                                                                              'et_reference', 
                                                                              'et_fraction'], 
                                             interp_method=interp_method, interp_days=interp_days,)

        
    #Fastest way to run et. This selects et. Could also select et_reference or et_fraction 
        monthly_df[start_date]=makeTable(monthly_coll.select(['et']).sum().reproject(crs=study_crs, scale=30).reduceRegion(reducer=ee.Reducer.median(),geometry= study_area,scale=30).getInfo())
        
        
    #These next three lines are the same as the one above, but broken into steps. Runs just barely slower on average
        #outimg = ee.Image(monthly_coll.select(['et']).sum()).reproject(crs=study_crs, scale=30).set('system:time_start',start_date)#.clip(study_area)
        #processOutImg = outimg.reduceRegion(reducer=ee.Reducer.median(),geometry= study_area,scale=30)#deleted crs
        #monthly_df[start_date]=makeTable(processOutImg.getInfo())
        
        
        print(monthly_df[start_date])
        print(start_date, end_date)
        

#check table
#print(monthly_df)
#print(pd.concat(monthly_df))


#Save et file 
pd.concat(monthly_df).to_csv(f"G:/My Drive/Bryce_EarthEngine/SSEBOP_Output/MonthlyBryceET.csv")


#print(processImgCol.getInfo())

           et
et  20.628789
2000-01-01 2000-02-01
           et
et  32.625061
2000-02-01 2000-03-01
           et
et  62.753291
2000-03-01 2000-04-01
           et
et  73.499635
2000-04-01 2000-05-01
           et
et  78.497311
2000-05-01 2000-06-01
            et
et  112.523415
2000-06-01 2000-07-01
          et
et  68.50083
2000-07-01 2000-08-01
           et
et  38.498479
2000-08-01 2000-09-01
           et
et  35.496774
2000-09-01 2000-10-01
           et
et  22.746891
2000-10-01 2000-11-01
           et
et  44.363705
2000-11-01 2000-12-01
           et
et  33.626685
2000-12-01 2001-01-01
           et
et  29.612013
2001-01-01 2001-02-01
           et
et  42.222693
2001-02-01 2001-03-01
           et
et  74.262815
2001-03-01 2001-04-01
           et
et  95.501031
2001-04-01 2001-05-01
           et
et  38.499072
2001-05-01 2001-06-01
           et
et  76.497488
2001-06-01 2001-07-01
           et
et  75.499395
2001-07-01 2001-08-01
           et
et  47.502898
2001-08-01 2001-09-01


           et
et  60.741024
2013-10-01 2013-11-01
           et
et  38.874415
2013-11-01 2013-12-01
           et
et  36.366639
2013-12-01 2014-01-01
           et
et  47.258602
2014-01-01 2014-02-01
           et
et  36.748431
2014-02-01 2014-03-01
           et
et  27.247751
2014-03-01 2014-04-01
           et
et  29.496078
2014-04-01 2014-05-01
           et
et  53.495024
2014-05-01 2014-06-01
           et
et  23.494737
2014-06-01 2014-07-01
           et
et  43.494313
2014-07-01 2014-08-01
           et
et  48.497152
2014-08-01 2014-09-01
           et
et  47.252377
2014-09-01 2014-10-01
           et
et  42.251228
2014-10-01 2014-11-01
           et
et  25.249986
2014-11-01 2014-12-01
           et
et  15.128184
2014-12-01 2015-01-01
           et
et  34.371523
2015-01-01 2015-02-01
          et
et  33.25176
2015-02-01 2015-03-01
           et
et  55.746634
2015-03-01 2015-04-01
          et
et  21.49906
2015-04-01 2015-05-01
           et
et  33.503711
2015-05-01 2015-06-01
    

# Convert csv for SWAT calibration (monthly and annual)

## Annual works great; montly needs to have the index removed to work, which I couldn't figure out and did manually with Excel - Kathryn

In [86]:
#add header and remove column that says "et"
monthly_et = pd.read_csv(f"G:/My Drive/Bryce_EarthEngine/SSEBOP_Output/MonthlyBryceET.csv", names=["Date","ET"], 
                         skiprows=1, usecols=[0,2])


#Convert to date and sort
monthly_et['Date'] = pd.to_datetime(monthly_et['Date']) #, format="%Y-%m-%d")
##monthly_et = monthly_et.groupby(pd.Grouper(key='Date', freq='1m'))
##monthly_et = monthly_et.sort_values(by='Date')
##monthly_et['Date'] = monthly_et['Date'].dt.strftime('%d/%m/%Y')
##monthly_et_final = monthly_et.reset_index(drop=True, inplace=True) #-not working and maybe not necessary to remove integer index

monthly_et.to_csv(f"G:/My Drive/Bryce_EarthEngine/SSEBOP_Output/BryceET_monthly.csv")


#summarize data by year and save
annual_et = monthly_et.groupby(pd.Grouper(key='Date', freq='1Y')).sum()
annual_et.to_csv(f"G:/My Drive/Bryce_EarthEngine/SSEBOP_Output/BryceET_annual.csv")

None


TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Index'